## LIBRARY SYSTEM
Features:
- Able to search for books by its title, content, genre.
- You can describe the characters or plot of the story, get the book.

Steps:
1. Import data from data files.
2. Vectorise it.
3. Create AI Agent for it.
4. Use the query to get answers.

Needed libraries:
1. langchain-text-splitters 
2. langchain-community 
3. langgraph
4. ebooklib
5. pip
6. ipykernel

Other Notes:
1. This is run inside conda environment.

In [ ]:
import os
import bs4
import ebooklib
from ebooklib import epub
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import WebBaseLoader
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# LLM SETUP
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
llm = init_chat_model("o3-mini", model_provider="openai")

### Load data (accepted PDF, Epub)

In [5]:
folder = "../data/books/"
list_of_books = os.listdir(folder)
list_of_books

['Julia_Quinn_-_Bridgertons__2_The_Viscount_Who_Loved_Me.epub',
 'Days at The Morisaki Bookshop 2.epub',
 'Julia_Quinn_-_Bridgertons__4_Romancing_Mister_Bridgerton.epub',
 'Ayesha_At_Last_-_Uzma_Jalaluddin.epub',
 'Julia_Quinn_-_Bridgertons__1_The_Duke_and_I.epub',
 '3. Before Your Memory Fades.epub',
 'Picking_Daisies_on_Sundays_-_Liana_Cincotti.epub',
 'What You Are Looking For is in The Library 2.epub',
 '1. Before The Coffee Gets Cold.epub',
 'Julia_Quinn_-_Bridgertons__5_To_Sir_Phillip_With_Love.epub',
 'Welcome To The Hyunam-dong Bookshop - Hwang Bo Reum 2.epub',
 'Secrets of Divine Love 2.epub',
 '09-Happily_Ever_After.pdf',
 'Because_of_Miss_Bridgerton_-_Rokesbys_1_-_Julia_Quinn.epub',
 '2. Tales From The Cafe.epub',
 'The Kamogawa Food Detectives 2.epub',
 '4. Before We Say Goodbye.epub',
 'Twisted_Love___40_Twisted_01__41__-_Ana_Huang 2.epub',
 'Julia_Quinn_-_Bridgertons__3_An_Offer_From_a_Gentleman.epub']

### Prompts

In [ ]:
# customise our own LLM

template = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""
custom_rag_prompt = PromptTemplate.from_template(template)

# context is list of documents in which this LLM/Agent is about.